In [ ]:
%pip install transformers[torch] huggingface_hub datasets evaluate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 7.7 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Torchtext není k dispozici pro poslední verzi pytorch, budeme tedy využuívat něco jiného ...


In [1]:
from transformers import Trainer, BertForSequenceClassification
from datasets import load_from_disk

import torch
import base

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base.reset_seed()

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [12]:
train = load_from_disk('./data/sst2/train-logits')
eval = load_from_disk('./data/sst2/eval-logits')
test = load_from_disk('./data/sst2/test-logits')

In [13]:
base.reset_seed()

In [14]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#model = base.freeze_model(model)

In [16]:
training_args = base.get_training_args(output_dir="./results/bertTest", logging_dir="./logs/bertTest", batch_size=16, epochs=10)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.685000,0.656375,0.625000,0.627303,0.625989,0.624324


In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.5503220558166504,
 'eval_accuracy': 0.729818780889621,
 'eval_precision': 0.7301365146481426,
 'eval_recall': 0.7297872879393202,
 'eval_f1': 0.7297071927478798,
 'eval_runtime': 3.2443,
 'eval_samples_per_second': 561.299,
 'eval_steps_per_second': 8.939,
 'epoch': 10.0}

In [ ]:
base.reset_seed()

In [ ]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir="./results/bertTestDistil", logging_dir="./logs/bertTestDistil", remove_unused_columns=False, batch_size=16, epochs=10, temp=8, lambda_param=.8)

In [ ]:
trainer = base.ImageDistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,5.882700,5.032340,0.572248,0.606610,0.566652,0.525007
2,5.746600,4.928039,0.628440,0.629813,0.629157,0.628159
3,5.508700,4.630551,0.636468,0.643428,0.634177,0.629569
4,5.177400,4.399031,0.672018,0.673875,0.670855,0.670129


KeyboardInterrupt: 

In [ ]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 3.7383933067321777,
 'eval_accuracy': 0.7287204832509611,
 'eval_precision': 0.7296473524174107,
 'eval_recall': 0.728667274622199,
 'eval_f1': 0.7284157327794416,
 'eval_runtime': 3.2214,
 'eval_samples_per_second': 565.289,
 'eval_steps_per_second': 9.002,
 'epoch': 10.0}